In [4]:
import pandas as pd
df = pd.read_csv('top_rated_wines.csv')
df = df[df['variety'].notna()] # remove any NaN values as it blows up serialization
data = df.sample(700).to_dict('records') # Get only 700 records. More records will make it slower to index
len(data)

700

In [5]:
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer

In [6]:
encoder = SentenceTransformer('all-MiniLM-L6-v2') # Model to create embeddings

In [7]:
# create the vector database client
qdrant = QdrantClient(":memory:") # Create in-memory Qdrant instance

In [8]:
# Create collection to store wines
qdrant.recreate_collection(
    collection_name="top_wines",
    vectors_config=models.VectorParams(
        size=encoder.get_sentence_embedding_dimension(), # Vector size is defined by used model
        distance=models.Distance.COSINE
    )
)

True

In [9]:
# vectorize!
qdrant.upload_points(
    collection_name="top_wines",
    points=[
        models.PointStruct(
            id=idx,
            vector=encoder.encode(doc["notes"]).tolist(),
            payload=doc,
        ) for idx, doc in enumerate(data) # data is the variable holding all the wines
    ]
)

In [10]:
user_prompt = "Suggest me an amazing Malbec wine from Argentina"

In [11]:
# Search time for awesome wines!

hits = qdrant.search(
    collection_name="top_wines",
    query_vector=encoder.encode(user_prompt).tolist(),
    limit=3
)
for hit in hits:
  print(hit.payload, "score:", hit.score)

{'name': 'Catena Zapata Adrianna Vineyard Malbec 2004', 'region': 'Argentina', 'variety': 'Red Wine', 'rating': 97.0, 'notes': '"The single-vineyard 2004 Malbec Adrianna Vineyard from the Gualtallary district is inky purple with aromas of wood smoke, pencil lead, game, black cherry, and blackberry liqueur. Opulent, full-flavored, yet remarkably light on its feet, this medium to full-bodied Malbec is all about pleasure. It will certainly evolve for a decade but is hard to resist now. It is a fine test of one\'s ability to defer immediate gratification. When all is said and done, Catena Zapata is the Argentina winery of reference – the standard of excellence for comparing all others. The brilliant, forward-thinking Nicolas Catena remains in charge, with his daughter, Laura, playing an increasingly large role. The Catena Zapata winery is an essential destination for fans of both architecture and wine in Mendoza. It is hard to believe, given the surge in popularity of Malbec in recent year

In [12]:
# define a variable to hold the search results
search_results = [hit.payload for hit in hits]

In [29]:
# Now time to connect to the local large language model
#before runing this cell execute the llamafile model as an administrator and ake sure it is running in local host.
from openai import OpenAI
client = OpenAI(
    base_url="http://127.0.0.1:8081/v1", # "http://<Your api-server IP>:port/v1"
    api_key = "sk-no-key-required"
)



completion = client.chat.completions.create(
    model="LLaMA_CPP",
    messages=[
        {"role": "system", "content": "You are chatbot, a wine specialist. Your top priority is to help guide users into selecting amazing wine and guide them with their requests."},
        {"role": "user", "content": "Suggest me an amazing Malbec wine from Argentina"},
        {"role": "assistant", "content": str(search_results)}
    ]
)
print(completion.choices[0].message.content)

ChatCompletionMessage(content='I suggest you try Catena Zapata Adrianna Vineyard Malbec 2004 from Argentina', role='assistant', function_call=None, tool_calls=None)
